In [15]:
import os
import cv2
import numpy as np
import taichi as ti
import shutil
import math

ti.init()


num_particles = 2000
dim = 3
world_scale_factor = 1.0 / 100.0
dt = 1e-2
mass_inv = 1.0

positions = ti.Vector.field(dim, float, num_particles)
velocities = ti.Vector.field(dim, float, num_particles)
pos_draw = ti.Vector.field(dim, float, num_particles)
force = ti.Vector.field(dim, float, num_particles)
positions0 = ti.Vector.field(dim, float, num_particles)
radius_vector = ti.Vector.field(dim, float, num_particles)
colors = ti.Vector.field(3, float, num_particles)  
q_inv = ti.Matrix.field(n=3, m=3, dtype=float, shape=())

@ti.kernel
def init_particles():
    init_pos = ti.Vector([30.0, 40.0, 0.0])
    cube_size = 20
    spacing = 2
    num_per_row = int(cube_size // spacing) + 1
    num_per_floor = num_per_row * num_per_row
    for i in range(num_particles):
        floor = i // (num_per_floor)
        row = (i % num_per_floor) // num_per_row
        col = (i % num_per_floor) % num_per_row
        positions[i] = ti.Vector([col * spacing, floor * spacing, row * spacing]) + init_pos
        velocities[i] = ti.Vector([-2.0, 3.0, 1.0])
        colors[i] = ti.Vector([ti.random(), ti.random(), ti.random()])

@ti.kernel
def shape_matching():
    elastic_factor = 0.98
    collid = 0
    gravity = ti.Vector([0.0, -9.8, 0.0])
    for i in range(num_particles):
        positions0[i] = positions[i]
        f = gravity
        velocities[i] += mass_inv * f * dt
        positions[i] += velocities[i] * dt
        if positions[i].y < 0.0:
            positions[i] = positions0[i]
            positions[i].y = 0.0
            collid = 1

    c = ti.Vector([0.0, 0.0, 0.0])
    for i in range(num_particles):
        c += positions[i]
    c /= num_particles

    A = sum1 = ti.Matrix([[0.0] * 3 for _ in range(3)], ti.f32)
    for i in range(num_particles):
        sum1 += (positions[i] - c).outer_product(radius_vector[i])
    A = sum1 @ q_inv[None]

    R, _ = ti.polar_decompose(A)

    for i in range(num_particles):
        positions[i] = c + R @ radius_vector[i]
        if collid == 1:
            velocities[i] = elastic_factor * (positions[i] - positions0[i]) / dt
        else:
            velocities[i] =  (positions[i] - positions0[i]) / dt


@ti.kernel
def compute_radius_vector():
    center_mass = ti.Vector([0.0, 0.0, 0.0])
    for i in range(num_particles):
        center_mass += positions[i]
    center_mass /= num_particles
    for i in range(num_particles):
        radius_vector[i] = positions[i] - center_mass

@ti.kernel
def precompute_q_inv():
    res = ti.Matrix([[0.0] * 3 for _ in range(3)], ti.f64)
    for i in range(num_particles):
        res += radius_vector[i].outer_product(radius_vector[i])
    q_inv[None] = res.inverse()

@ti.kernel
def rotation(angle: ti.f32):
    theta = angle / 180.0 * math.pi
    R = ti.Matrix([
        [ti.cos(theta), -ti.sin(theta), 0.0],
        [ti.sin(theta), ti.cos(theta), 0.0],
        [0.0, 0.0, 1.0]
    ])
    for i in range(num_particles):
        positions[i] = R @ positions[i]

@ti.kernel
def world_scale():
    for i in range(num_particles):
        pos_draw[i] = positions[i] * world_scale_factor



def substep():
    shape_matching()


window = ti.ui.Window("rigidbody", (1280, 720), vsync=True)
canvas = window.get_canvas()
scene = ti.ui.Scene()
camera = ti.ui.make_camera()


camera.position(0.5, 1.0, 1.95)
camera.lookat(0.5, 0.3, 0.5)
camera.fov(55)

init_particles()
rotation(30)  
compute_radius_vector()
precompute_q_inv()

while window.running:
    if window.get_event(ti.ui.PRESS):
        if window.event.key == ti.ui.ESCAPE:
            window.destroy()
            break

    for i in range(int(0.05/dt)):
        substep()


    camera.track_user_inputs(window, movement_speed=0.03, hold_key=ti.ui.RMB)
    scene.set_camera(camera)


    scene.point_light(pos=(0, 1, 2), color=(1, 1, 1))
    scene.point_light(pos=(0.5, 1.5, 0.5), color=(0.5, 0.5, 0.5))
    scene.ambient_light((0.5, 0.5, 0.5))

    world_scale()
    scene.particles(pos_draw, radius=0.01, per_vertex_color=colors)


    canvas.scene(scene)
    window.show()

[Taichi] Starting on arch=x64


In [16]:
import os
import cv2
import numpy as np
import taichi as ti
import shutil
import math

ti.init()


num_particles = 2000
dim = 3
world_scale_factor = 1.0 / 100.0
dt = 1e-2
mass_inv = 1.0

positions = ti.Vector.field(dim, float, num_particles)
velocities = ti.Vector.field(dim, float, num_particles)
pos_draw = ti.Vector.field(dim, float, num_particles)
force = ti.Vector.field(dim, float, num_particles)
positions0 = ti.Vector.field(dim, float, num_particles)
radius_vector = ti.Vector.field(dim, float, num_particles)
colors = ti.Vector.field(3, float, num_particles)  
q_inv = ti.Matrix.field(n=3, m=3, dtype=float, shape=())

@ti.kernel
def init_particles():
    init_pos = ti.Vector([30.0, 40.0, 0.0])
    cube_size = 20
    spacing = 2
    num_per_row = int(cube_size // spacing) + 1
    num_per_floor = num_per_row * num_per_row
    for i in range(num_particles):
        floor = i // (num_per_floor)
        row = (i % num_per_floor) // num_per_row
        col = (i % num_per_floor) % num_per_row
        positions[i] = ti.Vector([col * spacing, floor * spacing, row * spacing]) + init_pos
        velocities[i] = ti.Vector([-2.0, 3.0, 1.0])
        colors[i] = ti.Vector([ti.random(), ti.random(), ti.random()])

@ti.kernel
def shape_matching():
    elastic_factor = 0.98
    collid = 0
    gravity = ti.Vector([0.0, -9.8, 0.0])
    for i in range(num_particles):
        positions0[i] = positions[i]
        f = gravity
        velocities[i] += mass_inv * f * dt
        positions[i] += velocities[i] * dt
        if positions[i].y < 0.0:
            positions[i] = positions0[i]
            positions[i].y = 0.0
            collid = 1

    c = ti.Vector([0.0, 0.0, 0.0])
    for i in range(num_particles):
        c += positions[i]
    c /= num_particles

    A = sum1 = ti.Matrix([[0.0] * 3 for _ in range(3)], ti.f32)
    for i in range(num_particles):
        sum1 += (positions[i] - c).outer_product(radius_vector[i])
    A = sum1 @ q_inv[None]

    R, _ = ti.polar_decompose(A)

    for i in range(num_particles):
        positions[i] = c + R @ radius_vector[i]
        if collid == 1:
            velocities[i] = elastic_factor * (positions[i] - positions0[i]) / dt
        else:
            velocities[i] =  (positions[i] - positions0[i]) / dt


@ti.kernel
def compute_radius_vector():
    center_mass = ti.Vector([0.0, 0.0, 0.0])
    for i in range(num_particles):
        center_mass += positions[i]
    center_mass /= num_particles
    for i in range(num_particles):
        radius_vector[i] = positions[i] - center_mass

@ti.kernel
def precompute_q_inv():
    res = ti.Matrix([[0.0] * 3 for _ in range(3)], ti.f64)
    for i in range(num_particles):
        res += radius_vector[i].outer_product(radius_vector[i])
    q_inv[None] = res.inverse()

@ti.kernel
def rotation(angle: ti.f32):
    theta = angle / 180.0 * math.pi
    R = ti.Matrix([
        [ti.cos(theta), -ti.sin(theta), 0.0],
        [ti.sin(theta), ti.cos(theta), 0.0],
        [0.0, 0.0, 1.0]
    ])
    for i in range(num_particles):
        positions[i] = R @ positions[i]

@ti.kernel
def world_scale():
    for i in range(num_particles):
        pos_draw[i] = positions[i] * world_scale_factor



def substep():
    shape_matching()


window = ti.ui.Window("rigidbody", (1280, 720), vsync=True)
canvas = window.get_canvas()
scene = ti.ui.Scene()
camera = ti.ui.make_camera()


camera.position(0.5, 1.0, 1.95)
camera.lookat(0.5, 0.3, 0.5)
camera.fov(55)

init_particles()
rotation(30)  
compute_radius_vector()
precompute_q_inv()

frame_dir = "frame"
if not os.path.exists(frame_dir):
    os.makedirs(frame_dir)

frame_count = 0

while window.running:
    if window.get_event(ti.ui.PRESS):
        if window.event.key == ti.ui.ESCAPE:
            window.destroy()
            break

    for i in range(int(0.05/dt)):
        substep()


    camera.track_user_inputs(window, movement_speed=0.03, hold_key=ti.ui.RMB)
    scene.set_camera(camera)


    scene.point_light(pos=(0, 1, 2), color=(1, 1, 1))
    scene.point_light(pos=(0.5, 1.5, 0.5), color=(0.5, 0.5, 0.5))
    scene.ambient_light((0.5, 0.5, 0.5))

    world_scale()
    scene.particles(pos_draw, radius=0.01, per_vertex_color=colors)


    canvas.scene(scene)
    window.show()

    frame_filename = os.path.join(frame_dir, f"frame_{frame_count:04d}.png")
    window.save_image(frame_filename)  # Save window content to the image file

    frame_count += 1



frame_files = sorted(os.listdir(frame_dir))
frame_list = []
for filename in frame_files:
    if filename.endswith(".png"):
        img = cv2.imread(os.path.join(frame_dir, filename))
        frame_list.append(img)


video_filename = "output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  
fps = 24 
height, width, _ = frame_list[0].shape
video_writer = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))


for frame in frame_list:
    video_writer.write(frame)


video_writer.release()

shutil.rmtree(frame_dir)

print("Video has been saved successfully and the frame folder has been removed.")

[Taichi] Starting on arch=x64
Video has been saved successfully and the frame folder has been removed.
